In [200]:
import os
import sys
import re
import numpy as np 
import pandas as pd
from pathlib import Path
import matplotlib as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')

dir_meta = Path('meta')
dir_meta.mkdir(exist_ok=True, parents=True)

print(pd.__version__)
print(np.__version__)

dir_data = Path('data')
print(list(dir_data.iterdir()))


1.1.5
1.19.5
[WindowsPath('data/210607_CSV_All_Species_based.csv'), WindowsPath('data/CA.csv'), WindowsPath('data/CA_TTRS.xlsx'), WindowsPath('data/MC.csv'), WindowsPath('data/origin.csv'), WindowsPath('data/SC_TTRS.xlsx'), WindowsPath('data/SJ.csv'), WindowsPath('data/ThermalTolerance-Range Size.xlsx'), WindowsPath('data/TW_TTRS.xlsx')]



#### 讀取資料

#### 資料格式對齊:

- 改用慈蔓整理的資料:
    - CA_TTRS.xlsx、SC_TTRS.xlsx、TW_TTRS.xlsx
    - 以物種為單位: "210607_CSV_All_Species_based.csv"    
---
- ~~讀取原始資料 "MC-SJ-CA_ThermalTolerance-Range Size.xlsx"~~
    - ~~資料過濾~~
        - 刪除沒有資料的欄位 
            - df.dropna(axis='columns',how='all', inplace=True)
        - 刪除沒有名稱的欄位 
            - df.loc[:, ~df_CA.columns.str.contains('^Unnamed')]
        - 刪除沒有檔名的資料 
            - df.dropna(subset=['No.'], how='any')
        - 刪除欄位名稱是超連結的欄位 
            - df.loc[:, ~df_CA.columns.str.contains('https:')]
    - ~~資料轉換~~
        - 欄位名稱統一
            - df.rename(columns={'ID_20210210':'Species', 'N_W_length':'W_length',	'N_B_length':'B_length'})
        - 增加Event欄位
            - df['No.'].apply(lambda str_: str_[:4])
        - 改變資料型態
            - 物件 > 浮點數
                -  df[['Dw(g)','W_length','B_length']].apply(pd.to_numeric, errors='coerce')

In [201]:
## Discarded 
# ttrs_raw = 'data/MC-SJ-CA_ThermalTolerance-Range Size.xlsx'

# col_sel = ['Location', 'Event', 'No', 'Family', 'Species', 'Sex', 'Altitude', 'Hot', 'Cold', 'Temp_avg', 'Dw(g)', 'W_length', 'B_length']


# def excel_file_preprocess(df:pd.DataFrame)->pd.DataFrame:
#     print(f'Datt size : {len(df)} | # of Cols : {len(df.columns)}')

#     df.rename(columns={
#         'Sample_No.':'No', 
#         'No.':'No', 
#         'ID_20210210':'Species', 
#         'N_W_length':'W_length', 
#         'N_B_length':'B_length', 
#         'RS_Weight':'Dw(g)'},
#               inplace=True)

#     if 'Event' not in df.columns:
#             df['Event'] = df['No'].apply(lambda x: str(x)[:4])
#     if df["Event"].str.startswith('MC').all():
#         df.columns = df.columns.str.replace(r'.1', '_1')
#         df.rename(columns={'Temp_avg ':'Temp_avg', 'Temp_avg':'Temp_avg_1'}, inplace=True)

#     df[['Dw(g)','W_length','B_length']] = df[['Dw(g)','W_length','B_length']].apply(pd.to_numeric, errors='coerce')

#     df.dropna(axis='columns',how='all', inplace=True)
#     df.dropna(subset=['No'], how='any', inplace=True)

#     df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#     df = df.loc[:, ~df.columns.str.contains('https:')]

#     print(f'Datt size : {len(df)} | # of Cols : {len(df.columns)}')

#     return df

In [236]:
# col_renmed = {'Sample_No.':'No',
#               'No.':'No', 
#             'Coll_datw':'Date',
#             'ID_20210210':'Species',
#             'Weight':'Dw(g)',
#             'N_W_length':'W_length'	,
#             'N_B_length':'B_length',
#             }

- MC

In [159]:
## Discarded 
# df_MC = pd.read_excel(ttrs_raw,
#                       header=0,
#                       engine='openpyxl',
#                     #   skiprows=list(range(1,16)),
#               sheet_name='MCTT&RS')

# df_MC = excel_file_preprocess(df_MC)

# df_MC['Location'] = 'Taiwan'
# df_MC = df_MC[col_sel]
# df_MC.to_csv('data/MC.csv')
# df_MC = pd.read_csv('data/MC.csv', index_col=0)
# print(f'Datt size : {len(df_MC)} | # of Cols : {len(df_MC.columns)}')

# # print(df_MC.info())
# print(df_MC.describe())
# df_MC


Datt size : 15338 | # of Cols : 40
Datt size : 15338 | # of Cols : 38


In [ ]:
## read Taiwan data
# df_MC_TTRS = pd.read_excel('data/TW_TTRS.xlsx', engine='openpyxl',
#                       header=0,
#                       usecols="A:Q",
#                       sheet_name='Sheet1'
#                         )

# df_MC_TTRS['Location'] = 'Taiwan'
# df_MC_TTRS.rename(columns=col_renmed, inplace=True)
# df_MC_TTRS.dropna(subset=['No'], how='any', inplace=True)
# df_MC_TTRS

- CA

In [ ]:
## Discarded 
# df_CA = pd.read_excel(ttrs_raw,
#                       header=0, 
#                       skiprows=list(range(1,16)),
#               sheet_name='CATT&RS')

# df_CA = excel_file_preprocess(df_CA)
# df_CA['Location'] = 'Malaysia'
# df_CA = df_CA[col_sel]
# df_CA.to_csv('data/CA.csv')
# df_CA = pd.read_csv('data/CA.csv', index_col=0)
# print(f'Datt size : {len(df_CA)}. | # of Cols : {len(df_CA.columns)}')

# col_CA = df_CA.columns
# print(col_CA)
# print(df_CA.info())
# print(df_CA.describe())
# df_CA

In [250]:
## read Malaysia data
# df_CA_TTRS = pd.read_excel('data/CA_TTRS.xlsx', engine='openpyxl',
#                       header=0,
#                       usecols="B:R",
#                       skiprows=list(range(1,16)),
#                       sheet_name='Sheet1'
#                         )
# df_CA_TTRS['Location'] = 'Malaysia'
# df_CA_TTRS.rename(columns=col_renmed, inplace=True)
# df_CA_TTRS.dropna(subset=['No'], how='any', inplace=True)
# df_CA_TTRS

- SJ

In [189]:
## Discarded 
# df_SJ = pd.read_excel(ttrs_raw, engine='openpyxl',
#                       index_col=0,
#                       header=0,
#                       usecols="A:AK",
#                     #   skiprows=list(range(1,16)),
#                     sheet_name='SJTT&RS')
# df_SJ = excel_file_preprocess(df_SJ)

# df_SJ['Location'] = 'China'
# df_SJ = df_SJ[col_sel]
# df_SJ.to_csv('data/SJ.csv')
# df_SJ = pd.read_csv('data/SJ.csv', index_col=0)
# print(f'Datt size : {len(df_CA)}. | # of Cols : {len(df_CA.columns)}')

# col_SJ = df_SJ.columns
# print(col_SJ)
# # print(df_SJ.info())
# print(df_SJ.describe())
# df_SJ

Datt size : 4502 | # of Cols : 36
Datt size : 4275 | # of Cols : 33


In [ ]:
## read China data
# df_SJ_TTRS = pd.read_excel('data/SC_TTRS.xlsx', engine='openpyxl',
#                       header=0,
#                       usecols="B:R",
#                     #   skiprows=list(range(1,16)),
#                       sheet_name='Sheet1'
#                         )
# df_SJ_TTRS['Location'] = 'China'
# df_SJ_TTRS.rename(columns=col_renmed, inplace=True)
# df_SJ_TTRS.dropna(subset=['No'], how='any', inplace=True)
# df_SJ_TTRS

- 檔案命名混亂
    - 以正規表達式抓取檔名來跟METADATA作關聯

#### 資量匯整合併

In [249]:
# col_sel = ['Location', 'No', 'Family', 'Species',  'Altitude', 'W_length', 'B_length', 'Dw(g)', 'CTmax', 'CTmin', 'Tmax', 'Tmin', 'Tmean', 'DTR', ]
# df_SJ, df_MC, df_CA = df_SJ_TTRS[col_sel], df_MC_TTRS[col_sel], df_CA_TTRS[col_sel]
# df_TTRS = pd.concat([df_CA, df_SJ, df_MC])
# df_TTRS.to_csv('data/df_TTRS.csv')
df_TTRS = pd.read_csv('data/df_TTRS.csv', index_col=0)
df_TTRS.info()
df_TTRS.describe()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 23968 entries, 0 to 15337
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Location  23968 non-null  object 
 1   No        23968 non-null  object 
 2   Family    21960 non-null  object 
 3   Species   19900 non-null  object 
 4   Altitude  23927 non-null  float64
 5   W_length  19680 non-null  float64
 6   B_length  20008 non-null  float64
 7   Dw(g)     14094 non-null  float64
 8   CTmax     3438 non-null   float64
 9   CTmin     3317 non-null   float64
 10  Tmax      23927 non-null  float64
 11  Tmin      23927 non-null  float64
 12  Tmean     23927 non-null  float64
 13  DTR       23927 non-null  float64
dtypes: float64(10), object(4)
memory usage: 2.7+ MB


,Altitude,W_length,B_length,Dw(g),CTmax,CTmin,Tmax,Tmin,Tmean,DTR
count,23927.000000,19680.000000,20008.000000,14094.000000,3438.000000,3317.000000,23927.000000,23927.000000,23927.000000,23927.000000
mean,1655.884524,16.964986,13.697371,0.036997,41.421699,1.764673,23.522821,16.103750,19.018534,7.419071
std,763.526337,6.618324,5.495528,0.056004,3.345201,3.514151,4.163725,3.400635,3.576715,1.729688
min,140.000000,4.850000,1.160000,0.000050,-2.100000,-7.000000,12.346008,7.417441,9.995032,3.428648
25%,1110.000000,12.370000,9.870000,0.008700,39.800000,-0.700000,20.931257,14.437735,16.983421,6.198790
50%,1536.000000,16.330000,12.920000,0.019410,41.700000,1.400000,23.613013,15.994448,18.644272,7.109581
75%,2039.000000,19.950000,16.412500,0.045015,43.400000,3.700000,26.629627,18.688307,21.788433,9.153825
max,4152.000000,87.850000,55.530000,0.990000,49.100000,42.700000,31.829795,22.818702,26.269734,10.669312


In [245]:
df_TTRS.groupby('Location').mean()


,Altitude,W_length,B_length,Dw(g),CTmax,CTmin,Tmax,Tmin,Tmean,DTR
Location,,,,,,,,,,
China,2341.643659,18.332910,14.660538,0.044011,41.850974,0.873897,22.943146,13.750741,17.187871,9.192405
Malaysia,1132.103824,13.949544,11.796889,0.021124,41.245991,2.660284,22.832543,17.596089,19.849244,5.236454
Taiwan,1612.148650,17.596535,14.066211,0.043876,41.151792,1.815826,23.878544,16.339590,19.294954,7.538954


In [246]:
df_sp_based.groupby('Location').mean()

,Sample Size,TTRS_Elev_up,TTRS_Elev_low,TTRS_Elev_mid,TTRS_RS,TTRS_W_length,TTRS_B_length,TTRS_Weight,TTRS_CTmax,TTRS_CTmin,TTRS_CTrange,TTRS_Tmax,TTRS_Tmin,TTRS_Tmean,TTRS_DTR
Location,,,,,,,,,,,,,,,
China,6.095238,2264.080357,1791.279762,2027.68006,472.800595,17.891089,14.309312,0.040103,42.124513,1.043026,41.207814,24.052720,14.644782,18.080844,9.407938
Malaysia,2.732166,1162.319066,994.237354,1078.27821,168.081712,13.120170,11.364688,0.019359,41.461537,2.737338,38.445990,23.199600,17.882645,20.171556,5.316955
Taiwan,12.023859,1858.044606,1346.986515,1602.51556,511.058091,18.245272,14.951575,0.057042,41.332484,1.809045,39.407495,23.964022,16.330847,19.326446,7.633176


- 檢視整理好的、species-based 資料

In [133]:
df_sp_based = pd.read_csv('data/210607_CSV_All_Species_based.csv')
df_sp_based

,Location,Family,Species,Sample Size,TTRS_Elev_up,TTRS_Elev_low,TTRS_Elev_mid,TTRS_RS,TTRS_W_length,TTRS_B_length,TTRS_Weight,TTRS_CTmax,TTRS_CTmin,TTRS_CTrange,TTRS_Tmax,TTRS_Tmin,TTRS_Tmean,TTRS_DTR
0,Malaysia,Geometridae,CA_Abaxas_sp2,2,1959,1959,1959.0,0,28.025000,17.925000,0.027650,35.100000,0.100000,35.000000,17.298785,13.297546,15.040596,4.001239
1,Malaysia,Erebidae,CA_AGA01,2,1372,509,940.5,863,31.835000,22.700000,0.139000,41.000000,10.900000,30.100000,23.906834,18.108418,20.623503,5.798416
2,Malaysia,Erebidae,CA_AGA02,2,1066,903,984.5,163,31.390000,22.430000,0.094690,41.700000,NaN,NaN,23.963156,19.042622,21.094452,4.920534
3,Malaysia,Erebidae,CA_AGA03,1,1800,1800,1800.0,0,32.450000,24.520000,0.145590,NaN,NaN,NaN,18.717044,14.311935,16.155921,4.405110
4,Malaysia,Erebidae,CA_AMATA01,17,1374,903,1138.5,471,14.821875,12.638235,0.011705,41.800000,2.600000,39.200000,23.575832,18.442713,20.619982,5.133119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,China,Drepanidae,SJ_THY15,32,3579,2000,2789.5,1579,17.041875,12.862187,0.026804,38.342857,-1.976923,40.319780,21.146612,12.433047,15.682796,8.713565
3174,China,Drepanidae,SJ_THY16,7,2856,1864,2360.0,992,19.010000,15.688571,0.054564,NaN,-1.450000,NaN,20.522450,10.974509,14.733828,9.547941
3175,China,Drepanidae,SJ_THY17,4,2856,2000,2428.0,856,19.702500,15.540000,0.061695,40.600000,4.100000,36.500000,20.047220,10.660688,14.423787,9.386532
3176,China,Uraniidae,SJ_URA01,13,3175,1864,2519.5,1311,13.691667,9.000833,0.008242,39.166667,-1.200000,40.366667,20.598435,10.588436,14.490011,10.009999


In [244]:
df_sp_based.groupby('Location').mean()

,Sample Size,TTRS_Elev_up,TTRS_Elev_low,TTRS_Elev_mid,TTRS_RS,TTRS_W_length,TTRS_B_length,TTRS_Weight,TTRS_CTmax,TTRS_CTmin,TTRS_CTrange,TTRS_Tmax,TTRS_Tmin,TTRS_Tmean,TTRS_DTR
Location,,,,,,,,,,,,,,,
China,6.095238,2264.080357,1791.279762,2027.68006,472.800595,17.891089,14.309312,0.040103,42.124513,1.043026,41.207814,24.052720,14.644782,18.080844,9.407938
Malaysia,2.732166,1162.319066,994.237354,1078.27821,168.081712,13.120170,11.364688,0.019359,41.461537,2.737338,38.445990,23.199600,17.882645,20.171556,5.316955
Taiwan,12.023859,1858.044606,1346.986515,1602.51556,511.058091,18.245272,14.951575,0.057042,41.332484,1.809045,39.407495,23.964022,16.330847,19.326446,7.633176


In [147]:
df_taiwan = df_sp_based.query('Location == "Taiwan"')
print(df_taiwan.Species.nunique())
df_taiwan.Species.unique()
# df_taiwan.groupby(['Family','Species']).count()[['Sample Size']]
df_taiwan.describe()


964


,Sample Size,TTRS_Elev_up,TTRS_Elev_low,TTRS_Elev_mid,TTRS_RS,TTRS_W_length,TTRS_B_length,TTRS_Weight,TTRS_CTmax,TTRS_CTmin,TTRS_CTrange,TTRS_Tmax,TTRS_Tmin,TTRS_Tmean,TTRS_DTR
count,964.000000,964.000000,964.000000,964.00000,964.000000,910.000000,933.000000,364.000000,213.000000,241.000000,158.000000,964.000000,964.000000,964.000000,964.000000
mean,12.023859,1858.044606,1346.986515,1602.51556,511.058091,18.245272,14.951575,0.057042,41.332484,1.809045,39.407495,23.964022,16.330847,19.326446,7.633176
std,30.161188,676.908930,685.794758,616.65735,579.633634,7.147961,6.091714,0.076000,2.539703,2.651763,2.855876,4.306000,3.199306,3.587122,1.265184
min,1.000000,343.000000,343.000000,343.00000,0.000000,5.850000,4.400000,0.000100,32.700000,-4.000000,24.700000,14.220765,8.023275,10.750687,5.513848
25%,1.000000,1511.000000,876.000000,1110.00000,0.000000,13.522813,10.761429,0.015040,39.750000,-0.100000,37.798125,20.956755,14.437735,16.983421,6.578053
50%,3.000000,1995.000000,1255.000000,1574.50000,351.000000,17.063958,14.025000,0.034064,41.180000,1.633333,39.546667,23.850630,16.597846,19.431593,7.133994
75%,9.000000,2350.000000,1995.000000,2039.00000,910.000000,21.398500,17.830000,0.065562,43.033333,3.500000,41.126250,27.842132,18.688307,22.090445,8.988897
max,422.000000,3140.000000,2993.000000,3050.00000,2650.000000,85.135000,46.634545,0.606600,48.000000,14.600000,45.150000,31.829795,22.818702,26.269734,9.962890


#### 建立檔名, 類群(科、種), TTRS(RR, RS)的關聯資料檔
- 用於後續Autoencoder分析
- 根據影像檔名抓出對應的物種代號
    - 檔名 : 透過'NO' 及 'Specie' 與TTRS的metadata作關聯
    - 檔案來源
        - CA、SJ
            - 從檔案名抓出對應的'NO.'格式的名稱，再關聯到metadata
        - MC
            - 從已整理好的tag檔   
        
    

In [202]:
df_origin = pd.read_csv('data/origin.csv',sep='\t', header=None,  names=['fname'])
df_origin['fname'] = df_origin.fname.str.replace('_cropped.png', '')
df_origin


,fname
0,Aberrasine lichenshihi_male_FBG_10-IV-2015_SWu...
1,Abraxas adilluminata_female_Biluxi_1-VII-2015_...
2,Abraxas adilluminata_female_Hewang_28-X-2015_d...
3,Abraxas adilluminata_female_MTTS1558_1
4,Abraxas adilluminata_female_MTTS302_1
...,...
8916,Xestia supravidua_male_MCE2750_24-25-VII-2015_...
8917,Xestia supravidua_PT_female_Taiwan_800px
8918,Xestia tamsi_NSMT2015_180_1_800
8919,Yashmakia suffusa_male_Wulai_15-III-2014_SWu&W...


- CA&SJ

In [193]:
# [\d]* > 1或多個數字
pattern = r"(SJTT[\d]*|CATT[\d]*|CARS[\d]*|SJRS[\d]*)"
df_origin['No'] = df_origin.fname.str.extract(pattern)

print(df_origin.query('No.isnull()').__len__())
print(df_origin.query('No.notnull()').__len__())

577
8344


In [194]:
df_ori_Taiwan = df_origin.query('No.isnull()')
df_ori_Taiwan

,fname,No
0,Aberrasine lichenshihi_male_FBG_10-IV-2015_SWu...,NaN
1,Abraxas adilluminata_female_Biluxi_1-VII-2015_...,NaN
2,Abraxas adilluminata_female_Hewang_28-X-2015_d...,NaN
3,Abraxas adilluminata_female_MTTS1558_1,NaN
4,Abraxas adilluminata_female_MTTS302_1,NaN
...,...,...
8916,Xestia supravidua_male_MCE2750_24-25-VII-2015_...,NaN
8917,Xestia supravidua_PT_female_Taiwan_800px,NaN
8918,Xestia tamsi_NSMT2015_180_1_800,NaN
8919,Yashmakia suffusa_male_Wulai_15-III-2014_SWu&W...,NaN


- MC

- 從之前比對的清單來抓物種名

In [195]:
df_tags = pd.read_csv('./dearlep_img_species/meta/tags_Taiwan_220105.csv', index_col=0)
print(df_tags.columns)
df_tags = df_tags[['Name','Family','Species']]
df_tags

Index(['Taxon', 'Sex', 'Name', 'Suborder', 'Family', 'Species'], dtype='object')


,Name,Family,Species
0,Axylia putris triseriata_male_800px,Noctuidae,Axylia putris triseriata
1,Diarsia cia_male_36mm_800px,Noctuidae,Diarsia cia
2,"Diarsia yoshimotoi Plante, 1994_male-2_800px",Noctuidae,Diarsia yoshimotoi
3,Diarsia yoshimotoi_male_40mm_800px,Noctuidae,Diarsia yoshimotoi
4,Aberrasine lichenshihi_male_FBG_10-IV-2015_SWu...,Erebidae,Aberrasine lichenshihi
...,...,...,...
4398,Zanclognatha yaeyamalis_male_Japan_NSMT2016_01,Erebidae,Zanclognatha yaeyamalis
4399,Zeuxippa formosa_male_HT_NSMT3745_dissected_800,Zygaenidae,Zeuxippa formosa
4400,"Zonoplusia ochreata (Walker, 1865)_male_Mt-Zho...",Noctuidae,Zonoplusia ochreata
4401,Zusidava serratilinea_male_Cilan_420_4-V-2018_...,Drepanidae,Leucodrepana serratilinea


In [150]:
df_ori_Taiwan_ = df_ori_Taiwan.merge(df_tags,
                left_on='fname',
                right_on='Name',
                how='left'
                )[['fname','Family','Species',]]
df_ori_Taiwan_.info()
df_ori_Taiwan_ 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 577 entries, 0 to 576
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   fname    577 non-null    object
 1   Family   504 non-null    object
 2   Species  500 non-null    object
dtypes: object(3)
memory usage: 18.0+ KB


,fname,Family,Species
0,Aberrasine lichenshihi_male_FBG_10-IV-2015_SWu...,Erebidae,Aberrasine lichenshihi
1,Abraxas adilluminata_female_Biluxi_1-VII-2015_...,Geometridae,Abraxas adilluminata
2,Abraxas adilluminata_female_Hewang_28-X-2015_d...,Geometridae,Abraxas adilluminata
3,Abraxas adilluminata_female_MTTS1558_1,Geometridae,Abraxas adilluminata
4,Abraxas adilluminata_female_MTTS302_1,Geometridae,Abraxas adilluminata
...,...,...,...
572,Xestia supravidua_male_MCE2750_24-25-VII-2015_...,Noctuidae,Xestia supravidua
573,Xestia supravidua_PT_female_Taiwan_800px,Noctuidae,Xestia supravidua
574,Xestia tamsi_NSMT2015_180_1_800,Noctuidae,Xestia tamsi
575,Yashmakia suffusa_male_Wulai_15-III-2014_SWu&W...,Geometridae,Yashmakia suffusa
